In [1]:
#corresponding_classes = {'001':'废品', '002' : '次等品' ,'003':'良品'}

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import time
import pickle
import logging
from PIL import Image
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import random_ops,control_flow_ops
from tensorflow.python.keras.initializers import he_normal
import cv2
import math
import sys
add_dir = os.path.abspath("/opt/DahuaTech/MVViewer/share/Python")
sys.path.append(add_dir)
from capture import *
from tqdm import tqdm

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.app.flags.DEFINE_boolean('random_flip', True, "Whether to random flip")
tf.app.flags.DEFINE_boolean('random_brightness', True, "whether to adjust brightness")
tf.app.flags.DEFINE_boolean('random_contrast', True, "whether to random constrast")

tf.app.flags.DEFINE_integer('charset_size', 3, "Choose the first `charset_size` character to conduct our experiment.")
tf.app.flags.DEFINE_integer('image_size', 416, "Needs to provide same value as in training.")
tf.app.flags.DEFINE_boolean('gray', True, "whether to change the rbg to gray")
tf.app.flags.DEFINE_integer('max_steps', 100000, 'the max training steps ')
tf.app.flags.DEFINE_integer('eval_steps', 50, "the step num to eval")
tf.app.flags.DEFINE_integer('save_steps', 2000, "the steps to save")

tf.app.flags.DEFINE_string('checkpoint_dir', '/media/xinje/New Volume/building_blocks/checkpoint/', 'the checkpoint dir')
tf.app.flags.DEFINE_string('train_data_dir', './data2/train/', 'the train dataset dir')
tf.app.flags.DEFINE_string('test_data_dir', './data2/test/', 'the test dataset dir')
tf.app.flags.DEFINE_string('log_dir', './log', 'the logging dir')

tf.app.flags.DEFINE_boolean('restore', False, 'whether to restore from checkpoint')
tf.app.flags.DEFINE_integer('epoch', 1, 'Number of epoches')
tf.app.flags.DEFINE_integer('batch_size', 16, 'Validation batch size')
tf.app.flags.DEFINE_string('mode', 'train', 'Running mode. One of {"train", "valid", "test"}')
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [3]:
def random_flip(image):

    rand_num = random.random()*2-1
    if(rand_num <(-1/3)):
        return cv2.flip(image, -1)
    elif(rand_num>(-1/3) and rand_num < (1/3)):
        return cv2.flip(image, 0)
    elif(rand_num>(1/3)):
        return cv2.flip(image, 1)
    else:
        return cv2.flip(image, rand_num)
    
def random_crop(image):
    rate = (random.random()+1)*0.1*0.5 #random crop 10% to 20%
    cropImg = image[int(image.shape[0]*rate):int(image.shape[0]*(1-rate)),int(image.shape[1]*rate):int(image.shape[1]*(1-rate))]
    return cropImg
    
def random_rotate_image(image):
    #random rotate
    (h,w) = image.shape[:2]
    center = (w//2,h//2)
    M = cv2.getRotationMatrix2D(center,random.random()*360,1.0)
    image = cv2.warpAffine(image,M,(w,h),borderMode = cv2.BORDER_REPLICATE)
    return image

def random_distort_image(image, hue=18, saturation=1.5, exposure=1.5):
    def _rand_scale(scale):
        scale = np.random.uniform(1, scale)
        return scale if (np.random.randint(2) == 0) else 1. / scale

    # determine scale factors
    dhue = np.random.uniform(-hue, hue)
    dsat = _rand_scale(saturation)
    dexp = _rand_scale(exposure)
    # convert RGB space to HSV space
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype('float')
    # change satuation and exposure
    image[:, :, 1] *= dsat
    image[:, :, 2] *= dexp
    # change hue
    image[:, :, 0] += dhue
    image[:, :, 0] -= (image[:, :, 0] > 180) * 180
    image[:, :, 0] += (image[:, :, 0] < 0) * 180
    
    # avoid overflow when astype('uint8')
    image[...] = np.clip(image[...], 0, 255)
    # convert back to RGB from HSV
    return cv2.cvtColor(image.astype('uint8'), cv2.COLOR_HSV2RGB)

def data_augmentation(image):
    image = random_crop(image)
    image = random_flip(image)
    image = random_rotate_image(image)
    image = random_distort_image(image, hue=18, saturation=1.5, exposure=1.5)
    return image

In [4]:
def create_file_list(data_dir):
    img_list = []
    #get the file name from dir 0
    for _,file_path,files in os.walk(data_dir+str(0)):
        for file in files:
            img_list.append([data_dir+str(0)+'/'+file,0])
    #get the file name from dir 1        
    for _,file_path,files in os.walk(data_dir+str(1)):
        for file in files:
            img_list.append([data_dir+str(1)+'/'+file,1])

    random.shuffle(img_list)
    return img_list

def read_data(img_list,batch_size,aug):
    num_batch = len(img_list)/batch_size
    count=0
    while(True):
        x_data = []
        y_data = []
        for i in range(batch_size):
            temp_index = i+count*batch_size
            temp_index %=len(img_list) 
            image = cv2.imread(img_list[temp_index][0])
            if aug:
                image = data_augmentation(image)
            image = image[:,:,::-1]
            image = image.astype(np.float32)
            image = cv2.resize(image,(FLAGS.image_size,FLAGS.image_size))
            image = image/255
            x_data.append(image)
            y_data.append(img_list[temp_index][1])
        count+=1
        x_data = np.array(x_data)
        yield x_data,y_data

In [5]:
# class DataIterator:
#     def __init__(self, data_dir):
#         # Set FLAGS.charset_size to a small value if available computation power is limited.
#         truncate_path = data_dir  +('%d' % FLAGS.charset_size)
#         #print(truncate_path)
#         self.image_names = []
#         for root, sub_folder, file_list in os.walk(data_dir):
#             print(root)
#             if root < truncate_path:
#                 self.image_names += [os.path.join(root, file_path) for file_path in file_list]
#         random.shuffle(self.image_names)
#         self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in self.image_names]
#         #print(self.image_names)
#         #print(len(self.image_names))

#     @property
#     def size(self):
#         return len(self.labels)

#     @staticmethod
#     def data_augmentation(images):
#         if FLAGS.random_flip:
#             images = tf.image.random_flip_up_down(images)
#             images = tf.image.random_flip_left_right(images)
#             images = tf.image.random_hue(images,max_delta=0.3)
#             images = tf.image.random_contrast(images, 0.8, 1.2)
#             images = tf.image.random_brightness(images, max_delta=0.3)
#             degree_angle = random.random()*360
#             angles = degree_angle *math.pi/180
#             images = tf.contrib.image.rotate(images,angles=angles,interpolation='BILINEAR')
#         return images

#     def input_pipeline(self, batch_size, num_epochs=None, aug=True):
#         images_tensor = tf.convert_to_tensor(self.image_names, dtype=tf.string)
#         labels_tensor = tf.convert_to_tensor(self.labels, dtype=tf.int64)
#         input_queue = tf.train.slice_input_producer([images_tensor, labels_tensor], num_epochs=num_epochs)

#         labels = input_queue[1]
#         #print(input_queue)
#         images_content = tf.read_file(input_queue[0])
#         images = tf.image.convert_image_dtype(tf.image.decode_bmp(images_content, channels=3), tf.float32)
#         if aug:
#             images = self.data_augmentation(images)
#         new_size = tf.constant([FLAGS.image_size, FLAGS.image_size], dtype=tf.int32)
#         images = tf.image.resize_images(images, new_size)
#         image_batch, label_batch = tf.train.shuffle_batch([images, labels], batch_size=batch_size, capacity=5000,
#                                                           min_after_dequeue=500)
#         return image_batch, label_batch


In [6]:
# class DataIterator2:
#     def __init__(self, data_dir):
#         truncate_path = data_dir  +('%d' % FLAGS.charset_size)
#         self.image_names = []
#         for root, sub_folder, file_list in os.walk(data_dir):
#             print(root)
#             if root < truncate_path:
#                 self.image_names += [os.path.join(root, file_path) for file_path in file_list]
#         random.shuffle(self.image_names)
#         self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in self.image_names]

#     @property
#     def size(self):
#         return len(self.labels)

#     @staticmethod
#     def data_augmentation(images):
#         if FLAGS.random_flip_up_down:
#             images = tf.image.random_flip_up_down(images)
#         if FLAGS.random_brightness:
#             images = tf.image.random_brightness(images, max_delta=0.3)
#         if FLAGS.random_contrast:
#             images = tf.image.random_contrast(images, 0.8, 1.2)
#         return images

#     def input_pipeline(self, batch_size, num_epochs=None, aug=True):
#         images_tensor = tf.convert_to_tensor(self.image_names, dtype=tf.string)
#         labels_tensor = tf.convert_to_tensor(self.labels, dtype=tf.int64)
#         input_queue = tf.train.slice_input_producer([images_tensor, labels_tensor], num_epochs=num_epochs)

#         labels = input_queue[1]
#         #print(input_queue)
#         images_content = tf.read_file(input_queue[0])
#         images_names = input_queue[0]
#         images = tf.image.convert_image_dtype(tf.image.decode_bmp(images_content, channels=3), tf.float32)
#         if aug:
#             images = self.data_augmentation(images)
#         new_size = tf.constant([FLAGS.image_size, FLAGS.image_size], dtype=tf.int32)
#         images = tf.image.resize_images(images, new_size)
#         image_batch, label_batch,file_name = tf.train.batch([images, labels,images_names], batch_size=batch_size, capacity=5000)
#         #file_name = tf.as_string(file_name)
#         return image_batch, label_batch,file_name



In [7]:
# class DataIterator3:
#     def __init__(self, data_dir):
#         truncate_path = data_dir  +('%d' % FLAGS.charset_size)
#         self.image_names = []
#         for root, sub_folder, file_list in os.walk(data_dir):
#             print(root)
#             if root < truncate_path:
#                 self.image_names += [os.path.join(root, file_path) for file_path in file_list]
#         random.shuffle(self.image_names)
#         self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in self.image_names]

#     @property
#     def size(self):
#         return len(self.labels)

#     @staticmethod
#     def data_augmentation(images):
#         if FLAGS.random_flip_up_down:
#             images = tf.image.random_flip_up_down(images)
#         if FLAGS.random_brightness:
#             images = tf.image.random_brightness(images, max_delta=0.3)
#         if FLAGS.random_contrast:
#             images = tf.image.random_contrast(images, 0.8, 1.2)
#         return images

#     def input_pipeline(self, batch_size, num_epochs=None, aug=True):
#         images_tensor = tf.convert_to_tensor(self.image_names, dtype=tf.string)
#         labels_tensor = tf.convert_to_tensor(self.labels, dtype=tf.int64)
#         input_queue = tf.train.slice_input_producer([images_tensor, labels_tensor], num_epochs=num_epochs)

#         labels = input_queue[1]
#         #print(input_queue)
#         images_content = tf.read_file(input_queue[0])
#         images_names = input_queue[0]
#         images = tf.image.convert_image_dtype(tf.image.decode_bmp(images_content, channels=3), tf.float32)
# #         if aug:
# #             images = self.data_augmentation(images)
#         new_size = tf.constant([FLAGS.image_size, FLAGS.image_size], dtype=tf.int32)
#         images = tf.image.resize_images(images, new_size)
#         image_batch, label_batch = tf.train.batch([images, labels], batch_size=batch_size, capacity=5000)
#         #file_name = tf.as_string(file_name)
#         return image_batch, label_batch

In [5]:
def fully_connected(prev_layer, num_units, is_training):
    layer = tf.layers.dense(prev_layer, num_units, use_bias=True, activation=None)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    return layer

In [6]:
def conv_layer_conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)

    pool_layer1 = tf.layers.max_pooling2d(conv_layer1_out,[2,2],strides=2)
    return pool_layer1

In [7]:
def conv_layer_2conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)

    pool_layer2 = tf.layers.max_pooling2d(conv_layer2_out,[2,2],strides=2)
    return pool_layer2


In [8]:
def conv_layer_3conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    conv_layer3_out = tf.nn.relu(conv_layer3_bn)
    pool_layer3 = tf.layers.max_pooling2d(conv_layer3_out,[2,2],strides=2)
    return pool_layer3

In [9]:
def conv_layer_4conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    conv_layer3_out = tf.nn.relu(conv_layer3_bn)
    
    conv_layer4 = tf.layers.conv2d(conv_layer3_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer4_bn = tf.layers.batch_normalization(conv_layer4, training=is_training)
    conv_layer4_out = tf.nn.relu(conv_layer4_bn)
    pool_layer4 = tf.layers.max_pooling2d(conv_layer4_out,[2,2],strides=2)
    return pool_layer4

In [10]:
def convolutional_block(layer,layer_depth,is_training):
    depth1,depth2,depth3 = layer_depth
    shortcut = layer
    conv_layer1 = tf.layers.conv2d(layer, depth1, [1,1], 2, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, depth2, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, depth3, [1,1], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    
    conv_temp = tf.layers.conv2d(shortcut,depth3,[1,1],2,'same')
    conv_temp = tf.layers.batch_normalization(conv_temp,training=is_training)
    conv_add = conv_temp+conv_layer3_bn
    conv_layer3_out = tf.nn.relu(conv_add)
#     pool_layer3 = tf.layers.max_pooling2d(conv_layer3_out,[2,2],strides=2)
    return conv_layer3_out

In [11]:
def identity_block(layer,layer_depth,is_training):
    depth1,depth2,depth3 = layer_depth
    shortcut = layer
    conv_layer1 = tf.layers.conv2d(layer, depth1, [1,1], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, depth2, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, depth3, [1,1], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    
#     conv_temp = tf.layers.conv2d(shortcut,depth3,[1,1],1,'same')
    conv_add = shortcut+conv_layer3_bn
    conv_layer3_out = tf.nn.relu(conv_add)

    return conv_layer3_out

In [16]:
# def capture_image():
#     cameraCnt, cameraList = enumCameras()
#     camera = cameraList[0]
#     nRet = openCamera(camera)

#     streamSourceInfo = GENICAM_StreamSourceInfo()
#     streamSourceInfo.channelId = 0
#     streamSourceInfo.pCamera = pointer(camera)

#     streamSource = pointer(GENICAM_StreamSource())
#     nRet = GENICAM_createStreamSource(pointer(streamSourceInfo), byref(streamSource))
#     nRet = setSoftTriggerConf(camera)

#     nRet = streamSource.contents.startGrabbing(streamSource, c_ulonglong(0), c_int(GENICAM_EGrabStrategy.grabStrartegySequential))
#     time.sleep(1)
#     nRet = grabOne(camera)  
#     if( nRet != 0 ):
#         print("grabOne fail!")
#         # 释放相关资源
#         streamSource.contents.release(streamSource)        
#     else:
#         print("trigger time: " + str(datetime.datetime.now()))

#     frame = pointer(GENICAM_Frame())
#     nRet = streamSource.contents.getFrame(streamSource, byref(frame), c_uint(1000))
#     if ( nRet != 0 ):
#         print("SoftTrigger getFrame fail! timeOut [1000]ms")
#         # 释放相关资源
#         streamSource.contents.release(streamSource)   
#     else:
#         print("SoftTrigger getFrame success BlockId = " + str(frame.contents.getBlockId(frame))) 
#         print("get frame time: " + str(datetime.datetime.now()))   

#     nRet = frame.contents.valid(frame)
#     if ( nRet != 0 ):
#         print("frame is invalid!")
#         # 释放驱动图像缓存资源
#         frame.contents.release(frame)
#         # 释放相关资源
#         streamSource.contents.release(streamSource)


#     imageSize = frame.contents.getImageSize(frame)
#     buffAddr = frame.contents.getImage(frame)
#     frameBuff = c_buffer(b'\0', imageSize)
#     memmove(frameBuff, c_char_p(buffAddr), imageSize)

#     convertParams = IMGCNV_SOpenParam()
#     convertParams.dataSize = imageSize
#     convertParams.height = frame.contents.getImageHeight(frame)
#     convertParams.width = frame.contents.getImageWidth(frame)
#     convertParams.paddingX = frame.contents.getImagePaddingX(frame)
#     convertParams.paddingY = frame.contents.getImagePaddingY(frame)
#     convertParams.pixelForamt = frame.contents.getImagePixelFormat(frame)
#     frame.contents.release(frame)

#     # 保存bmp图片  
#     bmpInfoHeader = BITMAPINFOHEADER() 
#     bmpFileHeader = BITMAPFILEHEADER()

#     uRgbQuadLen = 0
#     rgbQuad = (RGBQUAD * 256)() # 调色板信息
#     rgbBuff = c_buffer(b'\0', convertParams.height * convertParams.width * 3)

#     # 如果图像格式是 Mono8 不需要转码 
#     if convertParams.pixelForamt == EPixelType.gvspPixelMono8:
#         # 初始化调色板rgbQuad 实际应用中 rgbQuad 只需初始化一次
#         for i in range(0, 256):
#             rgbQuad[i].rgbBlue = rgbQuad[i].rgbGreen = rgbQuad[i].rgbRed = i;

#         uRgbQuadLen = sizeof(RGBQUAD) * 256    
#         bmpFileHeader.bfSize = sizeof(bmpFileHeader) + sizeof(bmpInfoHeader) + uRgbQuadLen + convertParams.dataSize
#         bmpInfoHeader.biBitCount = 8
#     else:
#         # 转码 => BGR24
#         rgbSize = c_int()
#         nRet = IMGCNV_ConvertToBGR24(cast(frameBuff, c_void_p), byref(convertParams), \
#                                      cast(rgbBuff, c_void_p), byref(rgbSize))

#         if ( nRet != 0 ):
#             print("image convert fail! errorCode = " + str(nRet))
#             # 释放相关资源
#             streamSource.contents.release(streamSource) 

#         bmpFileHeader.bfSize = sizeof(bmpFileHeader) + sizeof(bmpInfoHeader) + rgbSize.value
#         bmpInfoHeader.biBitCount = 24   

#     if convertParams.pixelForamt == EPixelType.gvspPixelMono8:

#         image_information = frameBuff
#     else: 
#         image_information = rgbBuff
#     nRet = streamSource.contents.stopGrabbing(streamSource)
#     nRet = closeCamera(camera)
#     streamSource.contents.release(streamSource)
#     return image_information

In [17]:
def resnet_50(layer):
    #stage1
    X = tf.layers.conv2d(layer,64,[7,7],2,'valid',use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    X = tf.layers.batch_normalization(X,training=is_training)
    X = tf.nn.relu(X)
    X = tf.layers.max_pooling2d(X,[3,3],strides=2)
#stage2    
    X = convolutional_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    
#stage3
    X = convolutional_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)

#stage4
    X = convolutional_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)

#stage5
    X = convolutional_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
#average_pooling
    X = tf.layers.average_pooling2d(X,[2,2],strides=2)
# Flatten the output from the convolutional layers
    orig_shape = X.get_shape().as_list()
    layer = tf.reshape(X, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
    layer_drop = tf.nn.dropout(layer,keep_prob)
    layer2 = fully_connected(layer_drop, 1000, is_training)
    layer_drop2 = tf.nn.dropout(layer2,keep_prob)
    logits = tf.layers.dense(layer_drop2, 2)

In [18]:
def resnet_101(input_layer):
#stage1
    X = tf.layers.conv2d(layer,64,[7,7],2,'valid',use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    X = tf.layers.batch_normalization(X,training=is_training)
    X = tf.nn.relu(X)
    X = tf.layers.max_pooling2d(X,[3,3],strides=2)
#stage2    
    X = convolutional_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    
#stage3
    X = convolutional_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
#stage4
    X = convolutional_block(X,[256,256,1024],is_training)
    for i in range(22):
        X = identity_block(X,[256,256,1024],is_training)

#stage5
    X = convolutional_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
#average_pooling
    X = tf.layers.average_pooling2d(X,[2,2],strides=2)
# Flatten the output from the convolutional layers
    orig_shape = X.get_shape().as_list()
    layer = tf.reshape(X, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])

    layer_drop = tf.nn.dropout(layer,keep_prob)
    layer2 = fully_connected(layer_drop, 1000, is_training)
    layer_drop2 = tf.nn.dropout(layer2,keep_prob)
    logits = tf.layers.dense(layer_drop2, 2)
#     l2_loss = tf.losses.get_regularization_loss()
    return logits

In [19]:
def vgg_16(layer):
    conv_layer1 = conv_layer_2conv(layer, 64, is_training)
    conv_layer2 = conv_layer_2conv(conv_layer1, 128, is_training)
    conv_layer3 = conv_layer_3conv(conv_layer2, 256, is_training)
    conv_layer4 = conv_layer_3conv(conv_layer3, 512, is_training)
    conv_layer5 = conv_layer_3conv(conv_layer4, 512, is_training)


     
    # Flatten the output from the convolutional layers
    orig_shape = conv_layer5.get_shape().as_list()
    layer = tf.reshape(conv_layer5, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
    keep_prob = tf.placeholder('float')
    layer_drop = tf.nn.dropout(layer,keep_prob)
    layer2 = fully_connected(layer_drop, 4096, is_training)
    layer_drop2 = tf.nn.dropout(layer2,keep_prob)
    layer3 = fully_connected(layer_drop2, 4096, is_training)
    layer_drop3 = tf.nn.dropout(layer3,keep_prob)
    logits = tf.layers.dense(layer_drop3, 2)

In [12]:
def train_ch(num_batches, batch_size, learning_rate):

    inputs = tf.placeholder(tf.float32, [None, FLAGS.image_size, FLAGS.image_size, 3])
    labels = tf.placeholder(tf.int64, [None])
    keep_prob = tf.placeholder('float')
    is_training = tf.placeholder(tf.bool)
    layer = inputs 


    #stage1
    X = tf.layers.conv2d(layer,64,[7,7],2,'valid',use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    X = tf.layers.batch_normalization(X,training=is_training)
    X = tf.nn.relu(X)
    X = tf.layers.max_pooling2d(X,[3,3],strides=2)
#stage2    
    X = convolutional_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    
#stage3
    X = convolutional_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)

#stage4
    X = convolutional_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)

#stage5
    X = convolutional_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
#average_pooling
    X = tf.layers.average_pooling2d(X,[2,2],strides=2)
# Flatten the output from the convolutional layers
    orig_shape = X.get_shape().as_list()
    layer = tf.reshape(X, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
    layer_drop = tf.nn.dropout(layer,keep_prob)
    layer2 = fully_connected(layer_drop, 1000, is_training)
    layer_drop2 = tf.nn.dropout(layer2,keep_prob)
    logits = tf.layers.dense(layer_drop2, 2)


    l2_loss = tf.losses.get_regularization_loss()
    model_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    model_loss = model_loss+l2_loss
    # Tell TensorFlow to update the population statistics while training
    global_step = tf.get_variable("step", [], initializer=tf.constant_initializer(0), trainable=False)
    rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps=1000, decay_rate=0.97, staircase=True)
#     rate = learning_rate
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_opt = tf.train.AdamOptimizer(rate).minimize(model_loss,global_step=global_step)

    # Create operations to test accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), labels), tf.float32))
    tf.summary.scalar('loss',model_loss)
    tf.summary.scalar('accuracy',accuracy)
    tf.summary.scalar('learning_rate',rate)
    merged_summary_op = tf.summary.merge_all()
    # Train and test the network
    
    print('Strart training:')
    with tf.Session() as sess:
        train_list = create_file_list('./data2/train/')
        test_list = create_file_list('./data2/test/')
        train_feeder = read_data(train_list,batch_size,True)
        test_feeder = read_data(test_list,batch_size,False)
        test_feeder_final = read_data(test_list,1,False)
#         train_feeder = DataIterator(data_dir='./data2/train/')
#         test_feeder = DataIterator3(data_dir='./data2/test/')
#         train_images, train_labels = train_feeder.input_pipeline(batch_size)
#         test_images, test_labels = test_feeder.input_pipeline(batch_size)
#         test_images_1, test_labels_1 = test_feeder.input_pipeline(1)
        train_writer = tf.summary.FileWriter(FLAGS.log_dir+'/train',sess.graph)
        test_writer = tf.summary.FileWriter(FLAGS.log_dir+'/test')
        saver = tf.train.Saver(max_to_keep =100)
        sess.run(tf.global_variables_initializer())
        
        ckpt = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
        if ckpt:
            saver.restore(sess,ckpt)
            print('restore from the checkpoint {0}'.format(ckpt))
            global_step = global_step+int(ckpt.split('-')[-1])
            
        time.sleep(1)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        try:
            #using progress bar
            for batch_i in range(int(num_batches/50)):
                # train this batch
                temp_bar = tqdm([i+1 for i in range(50)])
                for i in temp_bar:
                    [train_images_batch, train_labels_batch] = next(train_feeder)
                    _,loss,acc = sess.run([train_opt,model_loss, accuracy], feed_dict={inputs: train_images_batch, labels: train_labels_batch, is_training: True,keep_prob:0.5})
                    # Periodically check the validation or training loss and accuracy
                    temp_bar.set_description('Batch: {:>2}: Training loss: {:>3.5f}, Training accuracy: {:>3.5f}'.format(batch_i*50+i, loss, acc))
                    if (batch_i*50+i)%200 == 0:
    #                     test_images_batch, test_labels_batch = sess.run([test_images, test_labels])
                        [test_images_batch, test_labels_batch] =next(test_feeder)
                        loss, acc,summary_op_test = sess.run([model_loss, accuracy,merged_summary_op], feed_dict={inputs: test_images_batch,
                                                                      labels: test_labels_batch,
                                                                      is_training: False,keep_prob:1.0})
                        test_writer.add_summary(summary_op_test, batch_i*50+i)
                        saver.save(sess,os.path.join(FLAGS.checkpoint_dir,'CH-reco'),global_step = batch_i*50+i)
                        temp_bar.set_description(
                            'Batch: {:>2}: Validation loss: {:>3.5f}, Validation accuracy: {:>3.5f}'.format(batch_i*50+i, loss, acc))
                    elif (batch_i*50+i)%50 == 0:
                        loss, acc,summary_op_train = sess.run([model_loss, accuracy,merged_summary_op], feed_dict={inputs: train_images_batch, labels: train_labels_batch, is_training: False,keep_prob:1.0})
                        train_writer.add_summary(summary_op_train, batch_i*50+i)
#                         print('Batch: {:>2}: Training loss: {:>3.5f}, Training accuracy: {:>3.5f}'.format(batch_i, loss, acc))
                    elif (batch_i*50+i) ==(num_batches-1):
                        accuracy_temp=0.0
                        for batch_j in range(test_feeder.size):
                            [test_images_batch_1, test_labels_batch_1] = next(test_feeder_final)
    #                         test_images_batch_1, test_labels_batch_1 = sess.run([test_images_1, test_labels_1])
                            [lg,acc] = sess.run([logits,accuracy], feed_dict={inputs: test_images_batch_1,labels: test_labels_batch_1,is_training: False,keep_prob:1.0})
                            accuracy_temp +=acc
                            print(lg)
                            print('Current test accuracy:{:>3.5f}'.format(acc))
                        accuracy_final = accuracy_temp/test_feeder.size
                        print('Final test accuracy: {:>3.5f} in {:>1} samples'.format(accuracy_final,test_feeder.size))
                    
        except tf.errors.OutOfRangeError:
            print('error')
        coord.request_stop()
        coord.join(threads)
        sess.close()


In [ ]:
num_batches = 50000 # 迭代次数
batch_size = 32  # 批处理数量
learning_rate = 1e-6  # 学习率

tf.reset_default_graph()
with tf.Graph().as_default():
    train_ch(num_batches, batch_size, learning_rate)

Strart training:
INFO:tensorflow:Restoring parameters from /media/xinje/New Volume/building_blocks/checkpoint/CH-reco-49600
restore from the checkpoint /media/xinje/New Volume/building_blocks/checkpoint/CH-reco-49600


Batch: 50: Training loss: 0.31408, Training accuracy: 0.96875: 100%|██████████| 50/50 [01:07<00:00,  1.36s/it]
Batch: 100: Training loss: 0.28817, Training accuracy: 1.00000: 100%|██████████| 50/50 [01:01<00:00,  1.23s/it]
Batch: 150: Training loss: 0.28735, Training accuracy: 1.00000: 100%|██████████| 50/50 [01:01<00:00,  1.23s/it]
Batch: 200: Validation loss: 0.63089, Validation accuracy: 0.93750: 100%|██████████| 50/50 [01:04<00:00,  1.29s/it]
Batch: 250: Training loss: 0.29401, Training accuracy: 1.00000: 100%|██████████| 50/50 [01:01<00:00,  1.22s/it]
Batch: 300: Training loss: 0.29345, Training accuracy: 1.00000: 100%|██████████| 50/50 [01:00<00:00,  1.21s/it]
Batch: 350: Training loss: 0.33578, Training accuracy: 0.96875: 100%|██████████| 50/50 [01:00<00:00,  1.22s/it]
Batch: 400: Validation loss: 0.79781, Validation accuracy: 0.90625: 100%|██████████| 50/50 [01:03<00:00,  1.27s/it]
Batch: 450: Training loss: 0.36924, Training accuracy: 0.96875: 100%|██████████| 50/50 [01:01<00:

In [ ]:
from matplotlib import pyplot as plt
import cv2
import time

In [ ]:
def Inference():

    inputs = tf.placeholder(tf.float32, [None, FLAGS.image_size, FLAGS.image_size, 3])
    labels = tf.placeholder(tf.int64, [None])
    is_training = tf.placeholder(tf.bool)
    keep_prob = tf.placeholder('float')
    layer = inputs 
#stage1
    X = tf.layers.conv2d(layer,64,[7,7],2,'valid',use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    X = tf.layers.batch_normalization(X,training=is_training)
    X = tf.nn.relu(X)
    X = tf.layers.max_pooling2d(X,[3,3],strides=2)
#stage2    
    X = convolutional_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    X = identity_block(X,[64,64,256],is_training)
    
#stage3
    X = convolutional_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)
    X = identity_block(X,[128,128,512],is_training)

#stage4
    X = convolutional_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)
    X = identity_block(X,[256,256,1024],is_training)

#stage5
    X = convolutional_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
    X = identity_block(X,[512,512,2048],is_training)
#average_pooling
    X = tf.layers.average_pooling2d(X,[2,2],strides=2)
# Flatten the output from the convolutional layers
    orig_shape = X.get_shape().as_list()
    layer = tf.reshape(X, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
    layer_drop = tf.nn.dropout(layer,keep_prob)
    layer2 = fully_connected(layer_drop, 1000, is_training)
    layer_drop2 = tf.nn.dropout(layer2,keep_prob)
    logits = tf.layers.dense(layer_drop2, 2)

  
    
#     conv_layer1 = conv_layer_2conv(layer, 64, is_training)
#     conv_layer2 = conv_layer_2conv(conv_layer1, 128, is_training)
#     conv_layer3 = conv_layer_3conv(conv_layer2, 256, is_training)
#     conv_layer4 = conv_layer_3conv(conv_layer3, 512, is_training)
#     conv_layer5 = conv_layer_3conv(conv_layer4, 512, is_training)


     
#     # Flatten the output from the convolutional layers
#     orig_shape = conv_layer5.get_shape().as_list()
#     layer = tf.reshape(conv_layer5, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
#     keep_prob = tf.placeholder('float')
#     layer_drop = tf.nn.dropout(layer,keep_prob)
#     layer2 = fully_connected(layer_drop, 4096, is_training)
#     layer_drop2 = tf.nn.dropout(layer2,keep_prob)
#     layer3 = fully_connected(layer_drop2, 4096, is_training)
#     layer_drop3 = tf.nn.dropout(layer3,keep_prob)
#     logits = tf.layers.dense(layer_drop3, 2)



    # Create operations to test accuracy
#     predict = tf.argmax(logits, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), labels), tf.float32))
    print('Strart Inference:')
    with tf.Session() as sess:
        test_list = create_file_list('./data2/train/')
        test_feeder = read_data(test_list,1,False)
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())        
        ckpt = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
        if ckpt:
            saver.restore(sess,ckpt)
            print('restore from the checkpoint {0}'.format(ckpt))               
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        try:
            #using progress bar
            temp_acc = 0
            for batch_i in range(len(test_list)):
                # train this batch
                [test_images_batch, test_labels_batch] = next(test_feeder)
#                 train_images_batch, train_labels_batch = sess.run([train_images, train_labels])
                acc = sess.run(accuracy, feed_dict={inputs: test_images_batch,labels: test_labels_batch,is_training: False,keep_prob:1.0})
                # Periodically check the validation or training loss and accuracy
                temp_acc+=acc
                print('Test accuracy:{} in sample:{}'.format(acc,batch_i))
            print('Final test accuracy: {:>3.5f} in {:>1} samples'.format(temp_acc/len(test_list),len(test_list)))
                    
        except tf.errors.OutOfRangeError:
            print('error')
        coord.request_stop()
        coord.join(threads)
        sess.close()


In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default():
    Inference()

In [ ]:
ls

In [ ]:
test_feeder = DataIterator3(data_dir='./data2/test/')
test_images_1, test_labels_1 = test_feeder.input_pipeline(1)

In [ ]:
test_images_1.dtype